# Reef Check - urchin size/frequency data

At sites where a sufficient number of urchins are present, 100 individuals of both red and purple urchins are sized using calipers. This is done anywhere at the site and is not associated with a transect. Urchin surveys are performed once per year, during the fall survey only.

In [95]:
## Imports

import pandas as pd
import numpy as np
import random

from datetime import datetime # for handling dates

In [96]:
## Ensure my general functions for the MPA data integration project can be imported, and import them

import sys
sys.path.insert(0, "C:\\Users\\dianalg\\PycharmProjects\\PythonScripts\\MPA data integration")

import WoRMS # functions for querying WoRMS REST API

## Load data

In [97]:
## Load data

data = pd.read_csv('RCCA_urchin_size_data.csv')
print(data.shape)
data.head()

(3424, 12)


,Site,Day,Month,Year,SurveyDate,Classcode,Size,Amount,Lat,Lon,Depth_ft,Temp10m
0,120 Reef,8,10,2006,8-Oct-06,purple urchin,2,10,33.737919,-118.392014,NaN,15.0
1,120 Reef,8,10,2006,8-Oct-06,purple urchin,3,16,33.737919,-118.392014,NaN,15.0
2,120 Reef,8,10,2006,8-Oct-06,purple urchin,4,23,33.737919,-118.392014,NaN,15.0
3,120 Reef,8,10,2006,8-Oct-06,purple urchin,5,28,33.737919,-118.392014,NaN,15.0
4,120 Reef,8,10,2006,8-Oct-06,purple urchin,6,20,33.737919,-118.392014,NaN,15.0


## Create occurrence file

Here, it seems reasonable for the **event** to be the survey (e.g. site + survey date). The **occurrrences** can be the urchin observations. 

In [98]:
## Get site names w/o spaces for use in eventID

# Get a list of site names with spaces removed
site_names = [name.replace(' ', '') for name in data['Site']]

# Map site_names to sites
site_name_dict = dict(zip(data['Site'], site_names))
site_name_dict["Fry's Anchorage"] = 'FrysAnchorage'

 # Create SiteName column from Site column
data['SiteName'] = data['Site']
data['SiteName'].replace(site_name_dict, inplace=True)
data.head()

,Site,Day,Month,Year,SurveyDate,Classcode,Size,Amount,Lat,Lon,Depth_ft,Temp10m,SiteName
0,120 Reef,8,10,2006,8-Oct-06,purple urchin,2,10,33.737919,-118.392014,NaN,15.0,120Reef
1,120 Reef,8,10,2006,8-Oct-06,purple urchin,3,16,33.737919,-118.392014,NaN,15.0,120Reef
2,120 Reef,8,10,2006,8-Oct-06,purple urchin,4,23,33.737919,-118.392014,NaN,15.0,120Reef
3,120 Reef,8,10,2006,8-Oct-06,purple urchin,5,28,33.737919,-118.392014,NaN,15.0,120Reef
4,120 Reef,8,10,2006,8-Oct-06,purple urchin,6,20,33.737919,-118.392014,NaN,15.0,120Reef


Unlike previous RCCA data, these data do not include any sites that are not in the site table.

In [99]:
## Pad month and day as needed

paddedDay = ['0' + str(data['Day'].iloc[i]) if len(str(data['Day'].iloc[i])) == 1 else str(data['Day'].iloc[i]) for i in range(len(data['Day']))]
paddedMonth = ['0' + str(data['Month'].iloc[i]) if len(str(data['Month'].iloc[i])) == 1 else str(data['Month'].iloc[i]) for i in range(len(data['Month']))]

In [100]:
## Create eventID

eventID = [data['SiteName'].iloc[i] + '_' + str(data['Year'].iloc[i]) + paddedMonth[i] + paddedDay[i] for i in range(len(data['Site']))]
occ = pd.DataFrame({'eventID':eventID})

occ.head()

,eventID
0,120Reef_20061008
1,120Reef_20061008
2,120Reef_20061008
3,120Reef_20061008
4,120Reef_20061008


In [101]:
## Format dates and add eventDate

eventDate = [datetime.strptime(dt, '%d-%b-%y').date().isoformat() for dt in data['SurveyDate']]
occ['eventDate'] = eventDate
occ.head()

,eventID,eventDate
0,120Reef_20061008,2006-10-08
1,120Reef_20061008,2006-10-08
2,120Reef_20061008,2006-10-08
3,120Reef_20061008,2006-10-08
4,120Reef_20061008,2006-10-08


In [102]:
## Add datasetID

occ['datasetID'] = 'RCCA urchin size'
occ.head()

,eventID,eventDate,datasetID
0,120Reef_20061008,2006-10-08,RCCA urchin size
1,120Reef_20061008,2006-10-08,RCCA urchin size
2,120Reef_20061008,2006-10-08,RCCA urchin size
3,120Reef_20061008,2006-10-08,RCCA urchin size
4,120Reef_20061008,2006-10-08,RCCA urchin size


In [103]:
## Add locality and countryCode

# locality
occ['locality'] = data['Site']

# countryCode
occ['countryCode'] = 'US'
occ.head()

,eventID,eventDate,datasetID,locality,countryCode
0,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US
1,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US
2,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US
3,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US
4,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US


In [104]:
## Load site table to get lat, lon

filename = 'RCCA_site_table.csv'
sites = pd.read_csv(filename, usecols=range(7))

sites.head()

,Research_group,Site,CA_MPA_Name_Short,MPA_status,LTM_project_short_code,Latitude,Longitude
0,RCCA,Macklyn Cove,NaN,REF,LTM_Kelp_SRock,42.045155,-124.294724
1,RCCA,Pyramid Pt,Pyramid Point SMCA,MPA,LTM_Kelp_SRock,41.994801,-124.217308
2,RCCA,Flat Iron Rock,NaN,NaN,NaN,41.059425,-124.157829
3,RCCA,Trinidad,NaN,NaN,NaN,41.055000,-124.139999
4,RCCA,MacKerricher North,MacKerricher SMCA,MPA,LTM_Kelp_SRock,39.492823,-123.801990


**Note** that some lat, lon pairs were missing from the urchin size data, although the sites are listed in the site table, and the lon, lat for those same sites are listed elsewhere in the data. For example, the following query will show some missing values for the Aquarium site:

``` python
occ[occ['locality'] == 'Aquarium']
```

Given this, it seemed most reasonable to load the site table and populate lat, lons from there.

In [105]:
## Add rows to site table -- CAN BE DELETED WHEN SITE TABLE IS UPDATED ON DATAONE

sites_to_add = pd.DataFrame({'Research_group':['RCCA']*5,
                            'Site':['Cayucos', 'Hurricane Ridge', 'LA Federal Breakwater', 'Ocean Cove Kelper', 'Pier 400'],
                            'Latitude':[35.4408, 37.4701, 33.711899, 38.555119, 33.716301],
                            'Longitude':[-120.936302, -122.4796, -118.241997, -123.3046, -118.258003]})
sites = pd.concat([sites, sites_to_add])

In [106]:
## Merge to obtain decimalLat, decimalLon

# Merge
occ = occ.merge(sites[['Site', 'Latitude', 'Longitude']], how='left', left_on='locality', right_on='Site')

# Rename columns
occ.rename(columns={'Latitude':'decimalLatitude', 'Longitude':'decimalLongitude'}, inplace=True)

# Drop Site
occ.drop(columns='Site', inplace=True)
occ.head()

,eventID,eventDate,datasetID,locality,countryCode,decimalLatitude,decimalLongitude
0,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014
1,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014
2,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014
3,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014
4,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014


In [107]:
## Add coordinateUncertainty in Meters

occ['coordinateUncertaintyInMeters'] = 250

In [108]:
## Add occurrenceID

occ['occurrenceID'] = data.groupby(['Site', 'SurveyDate'])['Classcode'].cumcount()+1
occ['occurrenceID'] = occ['eventID'] + '_occ' + occ['occurrenceID'].astype(str)

occ.head()

,eventID,eventDate,datasetID,locality,countryCode,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,occurrenceID
0,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ1
1,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ2
2,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ3
3,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ4
4,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ5


In [109]:
## Get unique common names

names = data['Classcode'].unique()
names

array(['purple urchin', 'red urchin'], dtype=object)

In [110]:
## Load species table to obtain scientific names

filename = 'RCCA_invertebrate_lookup_table.csv'
species = pd.read_csv(filename, encoding='ansi')

print(species.shape)
species.head()

(32, 10)


,Kingdom,Phylum,Class,Order,Family,Genus,Species,Classcode,taxonomic_source,taxonomic_id
0,Animalia,Echinodermata,Asteroidea,Valvatida,Asterinidae,Patiria,miniata,bat star,www.marinespecies.org,382131
1,Animalia,Cnidaria,Anthozoa,Alcyonacea,Plexauridae,Muricea,fruticosa/californica,brown/golden gorgonian,www.marinespecies.org,177745
2,Animalia,Echinodermata,Holothuroidea,Synallactida,Stichopodidae,Parastichopus,californicus,CA sea cucumber,www.marinespecies.org,711954
3,Animalia,Arthropoda,Malacostraca,Decapoda,Palinuridae,Panulirus,interruptus,CA spiny lobster,www.marinespecies.org,382898
4,Animalia,Mollusca,Gastropoda,Littorinimorpha,Cypraeidae,Neobernaya,spadicea,chestnut cowry,www.marinespecies.org,580674


In [111]:
## Map scientific names to classcodes and create scientificName

# Create scientific name column in species
species['scientificName'] = species['Genus'] + ' ' + species['Species']

# Map scientific names to classcodes
subset = species[species['Classcode'].isin(names)].copy()
code_to_species_dict = dict(zip(subset['Classcode'], subset['scientificName']))

# Fix misspellings
code_to_species_dict['red urchin'] = 'Mesocentrotus franciscanus'

# Create scientificName
occ['scientificName'] = data['Classcode']
occ['scientificName'].replace(code_to_species_dict, inplace=True)
occ.head()

,eventID,eventDate,datasetID,locality,countryCode,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,occurrenceID,scientificName
0,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ1,Strongylocentrotus purpuratus
1,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ2,Strongylocentrotus purpuratus
2,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ3,Strongylocentrotus purpuratus
3,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ4,Strongylocentrotus purpuratus
4,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ5,Strongylocentrotus purpuratus


In [112]:
## Match species in WoRMS

name_id_dict, name_name_dict, name_taxid_dict = WoRMS.run_get_worms_from_scientific_name(occ['scientificName'].unique(), verbose_flag=True)

In [113]:
## Add scientific name-related columns

occ['scientificNameID'] = occ['scientificName']
occ['scientificNameID'].replace(name_id_dict, inplace=True)

occ['taxonID'] = occ['scientificName']
occ['taxonID'].replace(name_taxid_dict, inplace=True)
occ.head()

,eventID,eventDate,datasetID,locality,countryCode,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,occurrenceID,scientificName,scientificNameID,taxonID
0,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ1,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747
1,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ2,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747
2,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ3,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747
3,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ4,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747
4,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ5,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747


In [114]:
## Add final name-related columns

occ['nameAccordingTo'] = 'WoRMS'
occ['occurrenceStatus'] = 'present'
occ['basisOfRecord'] = 'HumanObservation'

occ.head()

,eventID,eventDate,datasetID,locality,countryCode,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,occurrenceID,scientificName,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord
0,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ1,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation
1,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ2,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation
2,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ3,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation
3,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ4,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation
4,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ5,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation


In [115]:
## Add individualCount

occ['individualCount'] = data['Amount']
occ.head()

,eventID,eventDate,datasetID,locality,countryCode,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,occurrenceID,scientificName,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord,individualCount
0,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ1,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,10
1,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ2,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,16
2,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ3,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,23
3,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ4,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,28
4,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ5,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,20


In [116]:
## Add depth

occ['minimumDepthInMeters'] = round(data['Depth_ft']*0.3048, 1)
occ['maximumDepthInMeters'] = round(data['Depth_ft']*0.3048, 1)
occ.head()

,eventID,eventDate,datasetID,locality,countryCode,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,occurrenceID,scientificName,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord,individualCount,minimumDepthInMeters,maximumDepthInMeters
0,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ1,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,10,NaN,NaN
1,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ2,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,16,NaN,NaN
2,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ3,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,23,NaN,NaN
3,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ4,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,28,NaN,NaN
4,120Reef_20061008,2006-10-08,RCCA urchin size,120 Reef,US,33.737919,-118.392014,250,120Reef_20061008_occ5,Strongylocentrotus purpuratus,urn:lsid:marinespecies.org:taxname:240747,240747,WoRMS,present,HumanObservation,20,NaN,NaN


**Note** that there are 1175 records where depth is missing.

```python
occ[occ['minimumDepthInMeters'].isna() == True]
```

## Save

In [117]:
## Save

occ.to_csv('RCCA_urchin_size_occurrence_20200810.csv', index=False, na_rep='NaN')

## Create MoF

I can include urchin sizes and temperature data in the MoF.

In [129]:
## Add eventID, occurrenceID and size

mof = pd.DataFrame({'eventID':occ['eventID']})
mof['occurrenceID'] = occ['occurrenceID']
mof['measurementType'] = 'test diameter' # CHECK THIS
mof['measurementValue'] = data['Size']
mof['measurementUnit'] = 'centimeters'
mof['measurementMethod'] = 'measured with calipers and rounded down to the nearest centimeter' # CHECK THIS

print(mof.shape)
mof.head()

(3424, 6)


,eventID,occurrenceID,measurementType,measurementValue,measurementUnit,measurementMethod
0,120Reef_20061008,120Reef_20061008_occ1,test diameter,2,centimeters,measured with calipers and rounded down to the...
1,120Reef_20061008,120Reef_20061008_occ2,test diameter,3,centimeters,measured with calipers and rounded down to the...
2,120Reef_20061008,120Reef_20061008_occ3,test diameter,4,centimeters,measured with calipers and rounded down to the...
3,120Reef_20061008,120Reef_20061008_occ4,test diameter,5,centimeters,measured with calipers and rounded down to the...
4,120Reef_20061008,120Reef_20061008_occ5,test diameter,6,centimeters,measured with calipers and rounded down to the...


**Note** that I've assumed that the size measurement is the diameter of the test (spines excluded). Also, that measurements were rounded to the nearest centimeter.

In [130]:
## Group temperature to event level 
## NOTE that there is always a single temperature per event, but grouping handles any instances of multiple temperatures that might arise in the future by averaging

# Add eventID column to data
data['eventID'] = occ['eventID']

# Aggregate (using np.mean)
event_mof = data.groupby(['eventID']).agg({
    'Temp10m':[lambda x: round(np.mean(x), 1)]
})

# Tidy
event_mof.reset_index(inplace=True)
event_mof.columns = event_mof.columns.droplevel()
event_mof.columns = ['eventID', 'Temp10m_mean']

event_mof.head()

,eventID,Temp10m_mean
0,120Reef_20061008,15.0
1,120Reef_20070909,13.0
2,120Reef_20080830,17.0
3,120Reef_20090620,NaN
4,120Reef_20100801,13.0


In [131]:
## Add temperature

temp = pd.DataFrame({'eventID':event_mof['eventID']})
temp['occurrenceID'] = np.nan
temp['measurementType'] = 'temperature'
temp['measurementValue'] = event_mof['Temp10m_mean']
temp['measurementUnit'] = 'degrees Celsius'
temp['measurementMethod'] = 'measured by dive computer at 10 m depth, or at the seafloor if shallower than 10 m'

# Drop events that lack temperature data
temp.dropna(subset=['measurementValue'], inplace=True)

temp.head()

,eventID,occurrenceID,measurementType,measurementValue,measurementUnit,measurementMethod
0,120Reef_20061008,NaN,temperature,15.0,degrees Celsius,"measured by dive computer at 10 m depth, or at..."
1,120Reef_20070909,NaN,temperature,13.0,degrees Celsius,"measured by dive computer at 10 m depth, or at..."
2,120Reef_20080830,NaN,temperature,17.0,degrees Celsius,"measured by dive computer at 10 m depth, or at..."
4,120Reef_20100801,NaN,temperature,13.0,degrees Celsius,"measured by dive computer at 10 m depth, or at..."
5,120Reef_20110917,NaN,temperature,16.0,degrees Celsius,"measured by dive computer at 10 m depth, or at..."


In [132]:
## Concatenate

mof = pd.concat([mof, temp])
print(mof.shape)
mof.head()

(3593, 6)


,eventID,occurrenceID,measurementType,measurementValue,measurementUnit,measurementMethod
0,120Reef_20061008,120Reef_20061008_occ1,test diameter,2.0,centimeters,measured with calipers and rounded down to the...
1,120Reef_20061008,120Reef_20061008_occ2,test diameter,3.0,centimeters,measured with calipers and rounded down to the...
2,120Reef_20061008,120Reef_20061008_occ3,test diameter,4.0,centimeters,measured with calipers and rounded down to the...
3,120Reef_20061008,120Reef_20061008_occ4,test diameter,5.0,centimeters,measured with calipers and rounded down to the...
4,120Reef_20061008,120Reef_20061008_occ5,test diameter,6.0,centimeters,measured with calipers and rounded down to the...


In [134]:
## Drop records where temperature is NaN

mof.dropna(subset=['measurementValue'], inplace=True)
mof.shape

(3593, 6)

## Save

In [136]:
## Save

mof.to_csv('RCCA_urchin_size_MoF_20200810.csv', index=False, na_rep='NaN')

## Questions

1. Some records are missing latitude and longitude, even though these are known and provided for other records from the same site.
2. I assume the size listed represents test size (spines excluded)?
3. I assume sizes are rounded to the nearest centimeter? **Sizes are rounded DOWN to the nearest centimeter.**

In [146]:
mof['measurementType'].unique()

array(['test diameter', 'temperature'], dtype=object)